In [1]:
from ray import tune
import os

In [2]:
pwd =  '/home/lijiahao/workbench/SAVE'
import sys
sys.path.append(pwd)

In [3]:
from exps.SAVE_mi_tune import do_train
import seaborn as sns

In [4]:
train_func_name = 'do_train'
ray_res_dir = '/home/lijiahao/ray_results'
proj_dates = sorted([x for x in os.listdir(ray_res_dir) if train_func_name in x])
proj_dates

['do_train_2024-10-02_21-52-00',
 'do_train_2024-10-02_22-06-12',
 'do_train_2024-10-02_22-06-45',
 'do_train_2024-10-02_22-08-04',
 'do_train_2024-10-02_22-11-03',
 'do_train_2024-10-02_22-13-20',
 'do_train_2024-10-02_22-14-28',
 'do_train_2024-10-02_22-16-02',
 'do_train_2024-10-02_22-16-57',
 'do_train_2024-10-02_22-18-04',
 'do_train_2024-10-02_22-25-44',
 'do_train_2024-10-02_22-26-39',
 'do_train_2024-10-02_22-27-40',
 'do_train_2024-10-02_22-29-47',
 'do_train_2024-10-02_22-31-12',
 'do_train_2024-10-02_22-32-10',
 'do_train_2024-10-02_22-34-13',
 'do_train_2024-10-02_22-34-49',
 'do_train_2024-10-02_22-38-00',
 'do_train_2024-10-06_18-35-01',
 'do_train_2024-10-06_18-36-09',
 'do_train_2024-10-06_18-36-43',
 'do_train_2024-10-06_19-35-38',
 'do_train_2024-10-06_19-37-14',
 'do_train_2024-10-06_19-42-17',
 'do_train_2024-10-06_19-57-23',
 'do_train_2024-10-06_20-03-12',
 'do_train_2024-10-06_20-04-34',
 'do_train_2024-10-09_12-42-58',
 'do_train_2024-10-09_12-44-55',
 'do_train

In [5]:
def get_tuning_res(
    experiment_path: str = "/home/ljh/ray_results/save_train_2023-07-22_09-47-11",
    trainable=None,
):
    restore_tuner = tune.Tuner.restore(experiment_path, trainable=trainable)
    resotre_grid = restore_tuner.get_results()
    return resotre_grid.get_dataframe()

In [6]:
train_func = do_train
exp_date = proj_dates[-12]
experiment_path = f"/home/lijiahao/ray_results/{exp_date}"
res_df = get_tuning_res(experiment_path=experiment_path, trainable=train_func)

The trainable will be overwritten - this should be done with caution: it's possible to supply an incompatible trainable, and there are no guarantees that the resumed experiment will continue successfully. If you encounter errors during training, ensure that you are passing in the same trainable that was passed into the initial `Tuner` object.


In [7]:
exp_date

'do_train_2024-10-20_16-41-58'

In [7]:
select_cols = 'loss config/enc_dim config/lr'.split()

In [ ]:
config_df = res_df[select_cols]

In [ ]:
# small lr make good res
ax = sns.heatmap(config_df.corr(), annot=True, fmt='.2f')

In [7]:
import pandas as pd
df = pd.read_csv('/home/lijiahao/workbench/SAVE/result/SAVE_1018_cov_recon_opt.csv')

In [11]:
df['Total'][0]

'0.6663333583676287'

### look into one experiment res

In [ ]:
import os
os.chdir('/home/lijiahao/workbench/sc-save/')

In [ ]:
def get_single_tune_exp_id(ray_res_dir = '/home/lijiahao/ray_results', exp_date:str ='save_train_2023-07-20_20-39-03', trainable_name:str='save_train' ):
    single_tune_res = os.path.join(ray_res_dir, exp_date)
    tune_examples = [f for f in os.listdir(single_tune_res) if os.path.isdir(os.path.join(single_tune_res, f))]
    path_id_dict = {}
    for example in tune_examples:
        if example.startswith(trainable_name):
            id = example[len(trainable_name):].split('_')[1]
            path_id_dict[id] = os.path.join(ray_res_dir, exp_date, example)
        else:
            print('not correct trainable name')
    return path_id_dict

import matplotlib.pyplot as plt
import json

def plot_eval_trend(res_jsons:list, eval_key:str ='clu_eval', x_lim=None):
    eval_res = [j[eval_key] for j in res_jsons]
    if x_lim is not None:
        plt.xlim(x_lim) 
    plt.ylabel(eval_key)
    plt.xlabel('iteration')
    plt.plot(eval_res)

def get_example_res(tune_dic:dict, exp_id:str):
    with open(os.path.join(tune_dic[exp_id], 'result.json'),'r', encoding='utf-8') as f:
        lines = f.readlines()
    res_jsons = [json.loads(l.strip()) for l in lines]
    return res_jsons

In [ ]:
tune_dic = get_single_tune_exp_id(exp_date=exp_date)

In [ ]:
tune_dic.keys()

In [ ]:
res_jsons = get_example_res(tune_dic=tune_dic,
                            exp_id='c7d1dbdc')
plot_eval_trend(res_jsons=res_jsons,
                eval_key='scib_eval',
                x_lim=[0,100])

In [ ]:
[j['scib_eval'] for j in res_jsons]

### record res

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
res = np.load('/home/lijiahao/workbench/sc-save/tune_data/lung_atlas_res_rec.npy')

In [ ]:
scib_res_df = pd.DataFrame(data=res, columns=['scib_res', 'recon_loss', 'kl_loss'])

In [ ]:
scib_res_df['total_loss'] = scib_res_df['recon_loss'] + scib_res_df['recon_loss']

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plot_data = scib_res_df.iloc[60:80,:]
epochs = plot_data.index
fig = plt.figure(figsize=(12,12))
ax = plt.subplot(4,1,1)
plt.plot(epochs, plot_data['scib_res'])
plt.ylabel('scib_res')

ax = plt.subplot(4,1,2)
plt.plot(epochs, plot_data['recon_loss'])
plt.ylabel('recon_loss')

ax = plt.subplot(4,1,3)
plt.plot(epochs, plot_data['kl_loss'])
plt.ylabel('kl_loss')


ax = plt.subplot(4,1,4)
plt.plot(epochs, plot_data['total_loss'])
plt.ylabel('total_loss')

In [ ]:
sns.heatmap(scib_res_df.iloc[:,:].corr(), annot=True, linewidths=.5)